In [32]:
import pretty_errors
import statsmodels.formula.api as smf
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from datetime import datetime
from scipy import stats
from IPython.display import IFrame
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # 默认为'last'
plt.rcParams['font.sans-serif'] = ['Microsoft Yahei']
plt.style.use("ggplot")
%matplotlib inline

<div class="jumbotron">
    <h1 class="display-1">回归预测</h1>
    <hr class="my-4">
    <p>主讲：李岩</p>
    <p>管理学院</p>
    <p>liyan@cumtb.edu.cn</p>
</div>

## 多元线性回归模型

- 一种预测建模技术
- 研究一个或多个自变量与**一个**因变量之间的显著关系
- 展示自变量对因变量的影响强度
- 通过已经建立的回归模型预测新的目标值

###  房价是多少？

#### 读入数据

In [3]:
bostonDf = pd.read_csv('./data/analysis/boston.csv')
bostonDf.head()
bostonDf.shape

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


(506, 14)

变量|含义
---|---
CRIM|犯罪率
ZN|住宅用地所占比例
INDUS|城镇中非住宅用地所占比例
CHAS|虚拟变量
NOX|环保指数
RM|每栋住宅的房间数
AGE|1940年以前建成的自住单位的比例
DIS|距离5个波士顿的就业中心的加权距离
RAD|距离高速公路的便利指数
TAX|每一万美元的不动产税率
PTRATIO|城镇中的教师学生比例
B|城镇中的黑人比例
LSTAT|地区中有多少房东属于低收入人群
MEDV|自住房屋房价中位数（即均价）

### 模型表述

- 假设数据集有$d$个特征，对于第$i$个样本，特征为集合$\{x_{1}^{(i)},x_{2}^{(i)},\cdots,x_{d}^{(i)}\}$，则该样本的预测结果$\hat{y}^{(i)}$可以表示为

$$
\hat{y}^{(i)}=w_1x_{1}^{(i)}+w_2x_{2}^{(i)}+\cdots+w_dx_{d}^{(i)}+b
\label{eq:lineareqn}
$$

- 第$i$个样本的所有特征用向量$\boldsymbol{x}_i\in\mathbb{R}^d$表示，所有权重用向量$\boldsymbol{w}\in \mathbb{R}^d$表示，则式\eqref{eq:lineareqn}可用向量表示为

$$
\hat{y}^{(i)}=\boldsymbol{w}^T\boldsymbol{x}^{(i)}+b
\label{eq:linearvec}
$$

- 线性模型的目标是求解**模型参数**（model parameters）
    - $\boldsymbol{w}$
    - $b$

### 模型求解

\begin{definition}\label{def:lossfun}
损失函数（loss function）：量化目标的实际值与预测值之间的差距。
\end{definition}

- 通常选择**非负数**作为损失，且**数值越小表示损失越小**，完美预测时的损失为0

- 对于线形模型\eqref{eq:linearvec}，真实值为$y^{(i)}$，预测值为$\hat{y}^{(i)}$，可以用**平方误差**函数作为损失函数

$$l^{(i)}(\boldsymbol{w}, b) = \frac{1}{2} \left(\hat{y}^{(i)} - y^{(i)}\right)^2.$$

- 为度量模型在整个数据集上的质量，需计算在训练集$n$个样本上的损失均值，即

$$L(\boldsymbol{w}, b) =\frac{1}{n}\sum_{i=1}^n l^{(i)}(\boldsymbol{w}, b) =\frac{1}{n} \sum_{i=1}^n \frac{1}{2}\left(\boldsymbol{w}^\top \boldsymbol{x}^{(i)} + b - y^{(i)}\right)^2.$$

- 线形模型\eqref{eq:linearmatrix}的优化目标为

$$\boldsymbol{w}^*, b^* = \operatorname*{argmin}_{\boldsymbol{w}, b}\  L(\boldsymbol{w}, b).
\label{eq:linearobj}
$$

### 数据预处理

##### 重复值

In [4]:
bostonDf.duplicated().unique()

array([False])

- 结论：该数据集没有重复记录。

##### 缺失值

In [5]:
bostonDf.isna().sum(axis=0)

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

- 该数据集没有缺失值。

### sklearn建模

#### 划分训练集与检验集

In [6]:
from sklearn.model_selection import train_test_split
bsTrainX, bsTestX, bsTrainY, bsTestY = train_test_split(bostonDf.iloc[:,:bostonDf.shape[1]-1],bostonDf['MEDV'],test_size=0.3,random_state=100)

#### 建立模型

In [7]:
from sklearn import linear_model
bstLearModel = linear_model.LinearRegression()
bstLearModel

LinearRegression()

#### 训练模型

In [8]:
bstLearModel.fit(bsTrainX,bsTrainY)

LinearRegression()

##### 得到回归系数与截距

- 回归系数：`bstLearModel.coef_`
- 截距：`bstLearModel.intercept_`

In [9]:
print(f'回归系数为{[round(x,3) for x in bstLearModel.coef_]}')
print(f'截距为{bstLearModel.intercept_:.3f}')

回归系数为[-0.077, 0.04, -0.006, 2.63, -13.142, 3.844, -0.012, -1.376, 0.284, -0.014, -0.919, 0.011, -0.438]
截距为33.116


#### 预测

In [10]:
predicted = bstLearModel.predict(bsTrainX)
resDf = pd.DataFrame({'原价格':bsTrainY,'预测价格':predicted})
resDf

,原价格,预测价格
463,20.2,21.836585
75,21.4,23.495847
478,14.6,18.784720
199,34.9,29.445246
84,23.9,24.957573
...,...,...
343,23.9,27.180265
359,22.6,19.097343
323,18.5,19.494229
280,45.4,38.031242


### 拟合度

##### 决定系数$R^2$

$$R^2(y,\hat{y})=1-\frac{\sum_{i=1}^n(y_i-\hat{y}_i)^2}{\sum_{i=1}^n(y_i-\bar{y})^2}$$

其中，$\hat{y}_i$为预测值，$\bar{y}$为均值

In [11]:
# 训练集的R2
bstR2 = bstLearModel.score(bsTrainX,bsTrainY)
print(f'训练集的R-square为{bstR2:.3f}')

训练集的R-square为0.753


In [12]:
bstR2Test = bstLearModel.score(bsTestX,bsTestY)
print(f'检验集的R-square为{bstR2Test:.3f}')

检验集的R-square为0.706


##### 平均绝对误差 mean absolute error (MAE)

$$\rm{MAE}(y,\hat{y})=\frac{1}{n}\sum_{i=1}^n|y_i-\hat{y}_i|$$

In [13]:
from sklearn.metrics import mean_absolute_error
# 训练集的MAE
bstMAE = mean_absolute_error(bsTrainY,bstLearModel.predict(bsTrainX))
print(f'训练集的MAE为{bstMAE:.3f}')
# 检验集的MAE
bstMAETest = mean_absolute_error(bsTestY,bstLearModel.predict(bsTestX))
print(f'训练集的MAE为{bstMAETest:.3f}')

训练集的MAE为3.127
训练集的MAE为3.472


##### 平均绝对百分比误差 mean absolute percentage error (MAPE)

$$\rm{MAPE}(y,\hat{y})=\frac{1}{n}\sum_{i=1}^n\frac{|y_i-\hat{y}_i|}{\max(\epsilon,|y_i|)}$$

In [30]:
from sklearn.metrics import mean_absolute_percentage_error
# 训练集的MAE
bstMAPE = mean_absolute_percentage_error(bsTrainY,bstLearModel.predict(bsTrainX))
print(f'训练集的MAE为{bstMAPE:.3f}')
# 检验集的MAE
bstMAPETest = mean_absolute_percentage_error(bsTestY,bstLearModel.predict(bsTestX))
print(f'检验集的MAE为{bstMAPETest:.3f}')

训练集的MAE为0.157
检验集的MAE为0.171


##### 均方误差 mean squared error (MSE)

$$\rm{MSE}(y,\hat{y})=\frac{1}{n}\sum_{i=1}^n(y_i-\hat{y}_i)^2$$

In [15]:
from sklearn.metrics import mean_squared_error
# 训练集的MAE
bstMSE = mean_squared_error(bsTrainY,bstLearModel.predict(bsTrainX))
print(f'训练集的MAE为{bstMSE:.3f}')
# 检验集的MAE
bstMSETest = mean_squared_error(bsTestY,bstLearModel.predict(bsTestX))
print(f'训练集的MAE为{bstMSETest:.3f}')

训练集的MAE为19.067
训练集的MAE为29.799


## 逻辑回归

- logistic regression

- 因变量是**二元**变量(binary response)

### 线性回归的问题

<center>
    <img src='./img/analysis/linearRegBinaryRep.jpg' width=70%>
</center>

- 期望的预测结果

<center>
    <img src='./img/analysis/logistReg.jpg' width=70%>
</center>

### 逻辑回归的原理

令$\pi$表示事件{Y=1}发生的概率，$\mathbf{\beta}$为回归系数向量，则

$$
\pi = \frac{\rm{e}^{\mathbf{\beta^T X}}}{1+\rm{e}^{\mathbf{\beta^T X}}}=\frac{1}{1+\rm{e}^{-\mathbf{\beta^T X}}}
$$

- 上式即为sigmoid函数

- 转变为$\mathbf{\beta^T X}$的线性函数

$$
\log\frac{\pi}{1-\pi}=\mathbf{\beta^T X}
$$

- $\frac{\pi}{1-\pi}$被称为odds

- $\log\frac{\pi}{1-\pi}$被称为log odds

##### 极大似然估计参数（maximum likelihood）

- 优化模型参数以最大可能得到样本数据

- 即样本数据发生的概率最大

- 假设有$n$个样本，则样本发生的概率

$$
P\{Y_1=y_1,Y_2=y_2,\cdots,Y_n=y_n\}=P\{Y_1=y_1\}P\{Y_2=y_2\}\cdots P\{Y_n=y_n\}
$$

- 二元因变量服从二项分布

$$
\max \Pi_{i=1}^n\; \pi_i^{y_i}(1-\pi_i)^{1-y_i}
$$

令$L=\Pi_{i=1}^n\; \pi_i^{y_i}(1-\pi_i)^{1-y_i}$，则
$$
l = \log(L)=\sum_{i=1}^n\left(y_i\mathbf{\beta^T X_i}-\log(1+\rm{e}^{\mathbf{\beta^T X_i}})\right)
$$

- 逻辑回归系数含义
    - 令$\beta_i$为第$i$个自变量的回归系数

$$
\frac{\frac{\pi'}{1-\pi'}}{\frac{\pi}{1-\pi}}=\rm{e}^{\beta_i}
$$

- 上式左侧为odds ratio

### sklearn建模

#### 读入数据

In [16]:
# 读入泰坦尼克数据集

titanicDf = pd.read_csv('./data/analysis/train.csv',index_col=0)
titanicDf.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### 数值化分类属性

- one-hot编码：将分类属性转换成二元数值属性

```python
pandas.get_dummies(data, columns=None)
```
- `data`：`Series`类型，或者`DataFrame`类型
- `columns`：列名的`list`类型，数据集中哪些列需要转换，默认是将数据集中所有列进行转换
- 返回值：由二元化的属性构成的`DataFrame`

In [17]:
titDf = titanicDf.loc[:,~titanicDf.columns.isin(['Name','Ticket','Cabin'])].copy()
titDf.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S
2,1,1,female,38.0,1,0,71.2833,C
3,1,3,female,26.0,0,0,7.9250,S
4,1,1,female,35.0,1,0,53.1000,S
5,0,3,male,35.0,0,0,8.0500,S


In [18]:
titDf = pd.get_dummies(titDf,columns=['Pclass','Sex','Embarked'])
titDf.head()

,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,,
1,0,22.0,1,0,7.2500,0,0,1,0,1,0,0,1
2,1,38.0,1,0,71.2833,1,0,0,1,0,1,0,0
3,1,26.0,0,0,7.9250,0,0,1,1,0,0,0,1
4,1,35.0,1,0,53.1000,1,0,0,1,0,0,0,1
5,0,35.0,0,0,8.0500,0,0,1,0,1,0,0,1


#### 缺失值填充

In [19]:
titDf.fillna({'Age':titDf['Age'].mean()},inplace=True)

#### 划分训练集与检验集

In [20]:
titTrainX, titTestX, titTrainY, titTestY = train_test_split(titDf.loc[:,titDf.columns != 'Survived'],titDf['Survived'], random_state=100)

#### 建立并训练模型

In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
lrTit = LogisticRegression(C=1e9, max_iter=4000, solver='lbfgs', random_state=10)

- `C`：正则强度的倒数，降低过拟合
- `max_iter`：最大迭代次数
- `solver`：优化器，可选‘lbfgs’, ‘liblinear’, ‘newton-cg’, ‘newton-cholesky’, ‘sag’, ‘saga’

In [23]:
lrTit.fit(titTrainX,titTrainY)

LogisticRegression(C=1000000000.0, max_iter=4000, random_state=10)

In [24]:
lrTit.coef_   # 系数
lrTit.intercept_ # 截距

array([[-3.99012829e-02, -3.06012529e-01, -1.80748006e-01,
         3.77228100e-03,  2.31619974e+00,  1.57591901e+00,
         2.25501533e-01,  3.48650730e+00,  6.31112984e-01,
        -5.90427756e+00, -6.16288567e+00, -6.31720738e+00]])

array([4.11762037])

#### 计算odds ratio

In [25]:
np.exp(lrTit.coef_)

array([[9.60884290e-01, 7.36377393e-01, 8.34645658e-01, 1.00377941e+00,
        1.01370775e+01, 4.83518316e+00, 1.25295095e+00, 3.26716361e+01,
        1.87970149e+00, 2.72775171e-03, 2.10616679e-03, 1.80497709e-03]])

- `odds ratio > 1`, 该属性能<mark>提升</mark>被救的odds
- `odds ratio = 1`, 该属性不改变被救的odds
- `odds ratio < 1`, 该属性能<mark>降低</mark>被救的odds

#### 预测

##### 预测类别

In [26]:
titTestPred = lrTit.predict(titTestX)
testPredDf = pd.DataFrame({'Real':titTestY,'Pred':titTestPred})
testPredDf

,Real,Pred
PassengerId,,
206,0,1
45,1,1
822,1,0
459,1,1
796,0,0
...,...,...
154,0,0
289,1,0
245,0,0


##### 预测所属类别概率

In [27]:
titTestPredProb = lrTit.predict_proba(titTestX)
# 类别顺序与self.classes_一致
testPredProbDf = pd.DataFrame(titTestPredProb, columns=lrTit.classes_)
testPredProbDf

,0,1
0,0.215626,0.784374
1,0.281235,0.718765
2,0.915879,0.084121
3,0.287535,0.712465
4,0.817525,0.182475
...,...,...
218,0.963236,0.036764
219,0.834708,0.165292
220,0.890886,0.109114
221,0.374629,0.625371
